## DATA Extraction

In [3]:
'''
All Packages Required
'''
import fitz
import spacy
import re
import os
import numpy as np
from datetime import datetime
from dateutil import relativedelta
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.tag import pos_tag
import en_core_web_sm
from nltk.chunk import conlltags2tree, tree2conlltags
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import constants
import csv
import json
import time
from nltk.stem import PorterStemmer
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')

nlp = en_core_web_sm.load()

In [4]:
'''
This Function will take one pdf and then return a string of text
'''
def getData(filename):
    resumedata = []
    pdf_document = fitz.open(filename) #Open Pdf
    for current_page in range(len(pdf_document)): #loop pdf pages
        page = pdf_document.loadPage(current_page) #load Page
        textpage = page.getText("text")  #Get Text contents
        textpage = textpage.replace("",'').lower() # Lower and remove a ascii
#        textpage = textpage.replace("⚫",'').replace("❖",'').replace("➢",'')
        resumedata.append(textpage) # input into a list and join the list
#    gg = pdf_document.pageCount
    return ''.join(resumedata)

In [5]:
'''
Takes in the string of text and gather words that can be used in word cloud
'''
def wordcloud(text):
    stop_words = set(stopwords.words('english')) # Create stop words
    
    word_tokens = word_tokenize(text) #TOkenise the words
    
    filtered_sentence = [w for w in word_tokens if not w in stop_words] #Remove Stop words
    
    filtered_sentence = []
    for w in word_tokens:      #Remove stop words
        if w not in stop_words:
            filtered_sentence.append(w)
    new = ' '.join(filtered_sentence)
    new = new.strip()
    new = re.sub('[^\w\s]','',new)
    return new

In [6]:
'''
This function is use to match the name to the email if person entity doesnt work
it will match the email with name 
'''
def email_detail(text):

    #Pattens for Regex
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", text)

    #Email Regrex
    if email:
        try:
            emailtxt =  email[0].split()[0].strip(';') 
        except IndexError:
            return
    return emailtxt


In [7]:
'''
Find first noun chunk instead
'''
def extract_name(string):
    r1 = str(string)
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(r1)
    name = []
    for token in doc.noun_chunks: #loop through all the docs noun_chunks
        name.append(token)
        if name != None:    #Find the first token and break out
            break
    resumename = name[0]
    resumename = str(resumename)
    if "javier" in resumename:  #Adhoc solution
        resumename = resumename[1:-3].replace("\n","")
    return resumename


In [8]:
'''
Find the first Person label and match it with email
'''
def getname(text):
    doc = nlp(text)
    list(doc.sents)
    #get the person label
    entities = [e.string.lower() for e in doc.ents if 'PERSON'==e.label_]
    entities = list(entities)
    emailtxt = email_detail(text)
    emailtxt = emailtxt[0:3]
    #Check if email in first 3 is same as first 3 of name 
    for x in entities:
        if emailtxt in x:
            if "tan ling" in x:
                linglist = []
                x = linglist.append(x)
                x = linglist.append("ping")
                x = ''.join(linglist)
                linglist = linglist.clear()
            return x
        else:
            continue

In [9]:
'''
If first person label doesnt work it will call first noun chunk
'''
def final_names(text):
        if getname(text) == None:
            newname = extract_name(text)
            newname = newname.replace("\n",'').replace(' ','')
            return newname
        else:
            newname2 = getname(text)
            newname2 = newname2.replace('\n','').replace(' ','')
            return newname2

In [10]:
'''
Get Total_experience from a list of experience which is the total entities
it will call months from dates and return months of experience in months
'''

def get_total_experience(experience_list):
    '''
    Wrapper function to extract total months of experience from a resume
    :param experience_list: list of experience text extracted
    :return: total months of experience
    '''
    exp_ = []
    for line in experience_list:
        #regex to find the dates
        experience = re.search('(?P<fmonth>\w+.\d+)\s*(\D|to)\s*(?P<smonth>\w+.\d+|present)', line, re.I)
        if experience:
            exp_.append(experience.groups())
    total_experience_in_months = sum([get_number_of_months_from_dates(i[0], i[2]) for i in exp_])
    return total_experience_in_months


In [11]:
'''
Date calculation
'''
def get_number_of_months_from_dates(date1, date2):
    '''
    Helper function to extract total months of experience from a resume
    :param date1: Starting date
    :param date2: Ending date
    :return: months of experience from date1 to date2
    '''
    monAdd1 = ["Jan"," "]
    monAdd2 = ["Jan"," "]
    if date2.lower() == 'present':
        date2 = datetime.now().strftime('%b %Y')
#if date is like 2012 add "jan" for both
    if len(date1) == 4 and len(date2) == 4:
        monAdd1.append(date1)
        monAdd2.append(date2)
        date1 = ' '.join(monAdd1)
        date2 = ' '.join(monAdd2)
    
    try:
        if len(date1.split()[0]) > 3:
            date1 = date1.split()
            date1 = date1[0][:3] + ' ' + date1[1]            
            
        if len(date2.split()[0]) > 3:
            date2 = date2.split()
            date2 = date2[0][:3] + ' ' + date2[1]

    except IndexError:
        return 0
    try:
        date1 = datetime.strptime(str(date1), '%b %Y')
        date2 = datetime.strptime(str(date2), '%b %Y')
        months_of_experience = relativedelta.relativedelta(date2, date1)
        months_of_experience = months_of_experience.years * 12 + months_of_experience.months
#        if months_of_experience == 0:
#            months_of_experience = (int(yeardate2) - int(yeardate1)) * 12
    except ValueError:
        return 0
    return months_of_experience

In [12]:
'''
Function to extract all text below a set of words
'''
def extract_entity_sections_professional(text):
    '''
    :param text: Raw text of resume
    :return: dictionary of entities
    '''
    
    #double_checker default false if true will merge 2lines at a time so that dates range will
#not be split up
    double_checker = False
    
    text_split = [i.strip() for i in text.split('\n')]
    count=0
#Check if date range is split
    for i in text_split:
        if i[-1:] == "–":
            double_checker = True
#Special condition for jiaxin if there is campus experience remove word experience
        if "campus experience" in i:
            campus_check = text_split.index("campus experience")
            text_split[campus_check] = text_split[campus_check][:-11]
#If there is multiple occurance of education it will keep first and replace all others
        if 'education' in i:
#            print(i)
            count +=1
            educ = i
            if count > 1:
                number_list = text_split.index(educ)
#                print(number_list)
                text_split[number_list] = text_split[number_list].replace('education','')
#    print(text_split)
    entities = {}
    key = False
    for phrase in text_split:
        if len(phrase) == 1:
            p_key = phrase
        else:
            p_key = set(phrase.lower().split()) & set(constants.RESUME_SECTIONS_PROFESSIONAL)
        try:
            p_key = list(p_key)[0]
        except IndexError:
            pass
        if p_key in constants.RESUME_SECTIONS_PROFESSIONAL:
            entities[p_key] = []
            key = p_key
        elif key and phrase.strip():
            entities[key].append(phrase)
            
    return entities


In [13]:
'''
Personalized Extraction of entity function
'''
def extract_entity_sections_professional2(text):
    '''
    :param text: Raw text of resume
    :return: dictionary of entities
    '''
#double_checker default false if true will merge 2lines at a time so that dates range will
#not be split up
    double_checker = False
    
    text_split = [i.strip() for i in text.split('\n')]
    count=0
#Check if date range is split
    for i in text_split:
        if i[-1:] == "–":
            double_checker = True
#Special condition for jiaxin if there is campus experience remove word experience
        if "campus experience" in i:
            campus_check = text_split.index("campus experience")
            text_split[campus_check] = text_split[campus_check][:-11]
#If there is multiple occurance of education it will keep first and replace all others
        if 'education' in i:
            count +=1
            educ = i
            if count > 1:
                number_list = text_split.index(educ)
                text_split[number_list] = text_split[number_list].replace('education','')
    entities = {}
    key = False
    for phrase in text_split:
        if len(phrase) == 1:
            p_key = phrase
        else:
            p_key = set(phrase.lower().split()) & set(constants.RESUME_SECTIONS_PROFESSIONAL)
        try:
            p_key = list(p_key)[0]
        except IndexError:
            pass
        if p_key in constants.RESUME_SECTIONS_PROFESSIONAL:
            entities[p_key] = []
            key = p_key
        elif key and phrase.strip():
            entities[key].append(phrase)
#Return the dictionary where the key is experience only
        for k, v in entities.items():
            if k == "experience":
                gg = v
#If double_checker is true it will check if len of list is even, if not it will not work
#because zip will not print single item if not zipped
    if double_checker == True:
        if len(gg) % 2 != 0:
            gg.append("")
        g = [' '.join(x) for x in zip(gg[0::2], gg[1::2])]
        gg = g
    else:
#        print("This Document doesnt need to merge lines")
        pass
    
    return gg

In [14]:
'''
get email, education,number
'''

def alldetails(text):

    #Pattens for Regex
    EduPattern = ["National University of Singapore","NUS", "Nanyang Technological University","NTU"]
    phone = re.findall(re.compile(r'(?:(?:\+|\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([1-9]1[01-9]|[1-9][01-8]1|[1-9][01-8][01-9])\s*\)|([0-9][1-9]|[0-9]1[01-9]|[1-9][01-8]1|[1-9][01-8][01-9]))\s*(?:[.-]\s*)?)?([1-9]1[01-9]|[1-9][01-9]1|[1-9][0-9]{2})\s*(?:[.-]\s*)?([0-9]{5})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", text)
    #Phone Regrex
    if phone:
        number = ''.join(phone[0])
        if len(number) > 8:
            phone_num = ' '+'+' + number[:2] + '-' + number[:8]
        else:
            phone_num = ' '+'+65' + number[:0] + '-' + number[:8]

    #Email Regrex
    if email:
        try:
            email_name =  email[0].split()[0].strip(';')
        except IndexError:
            print("Error")

    #Regrex for Education
    EduPattern = [x.lower() for x in EduPattern]
    edu_list = []
    
    for pattern in EduPattern:
        if re.search(r'\b' + pattern + r'\b', text):
            if True:
                edu_list.append(pattern)
            else:
                continue
    if len(edu_list) == 0:
            edu_list.append('Nan')
    if "national university of singapore" and 'nus' in edu_list:
        remove_edu = edu_list.index('nus')
        edu_list.pop(remove_edu)
    if "national university of singapore" in edu_list:
        remove_edu1 = edu_list.index('national university of singapore')
        edu_list[remove_edu1] = 'nus'
    elif "nanyang technological university" in edu_list:
        remove_edu2 = edu_list.index('nanyang technological university')
        edu_list[remove_edu2] = 'ntu'
        
#    if "b.s." in edu_list:
#        no1 = edu_list.index("b.s.")
#        edu_list[no1] = "bachelor"
#    
#    elif ""
    edu_list = ','.join(edu_list)    
    res_detail1 = {'Phone Number':phone_num,'Email Address':email_name,'Education Level':edu_list}
    
    return res_detail1


In [15]:
'''
Extract skills
'''
def skills_extraction(text):

    nlp_text = nlp(text)
    #removing stop words and implement word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    #tokens = [token.for token in tokens]
    path = "C:/Users/TP_baseline/Documents/major_project/skills"
    # dirs = os.listdir(path)
    dictionary1 = {}
    skills_dict = {}
    with os.scandir(path) as now:
        for g in now:
            input_file = g
            read = pd.read_csv(input_file)
            skills = list(read.columns.values)
            dictionary1[str(input_file)] = skills
            #search all the csv file and matches the name
            skillset = []
            
            for token in tokens:
                if token.lower() in skills:
                    token = token.strip()
                    skillset.append(token)
            for token in nlp_text.noun_chunks:
#                print(token)
                token = token.text.lower().strip()
                if token in skills:
                    if "b.s" in token:
                        skillset.append("bachelor")
                    skillset.append(token)
            skillset = set(skillset)
            skillset = list(skillset)
            if len(skillset) == 0:
                skillset.append("Nan")
            skillset = ', '.join(skillset)
            skills_dict[str(input_file.name[:-4])] = skillset
                    
    return skills_dict

In [16]:
# For Rule based System more detailed the data Engineer criteria
def skills_point(skills_dict):

#    text = getData(filename).lower().strip()
#    skills_dict = skills_extraction(text)
    count3 = 0
    sd = skills_dict
    skill = sd['dataEn_skill']
    skill = skill.split(',')
    for s in skill:
        if "python" in s:
            count3  = count3 + 1
        if "java" in s:
            count3 = count3 + 1
        if "sql" in s:
            count3 = count3 + 1
    dE_no = (count3 * 3) + (len(skill) - count3)
    
    gth = sd['good_to_have'].strip()
    gth = gth.split(',')
    gth_no = len(gth)
    
    ra = sd['relevant_ability'].strip()
    ra = ra.split(',')
#    print(ra)
    ra_no = len(ra)
#    print(ra_no)
    
    ld = sd['leadership'].strip()
    ld = ld.split(',')
    ld_no = len(ld)
    
    rp = sd['results_performance'].strip()
    rp = rp.split(',')
    rp_no = len(rp)
    
    return dE_no,gth_no,ra_no,ld_no,rp_no
        

In [17]:
'''
To get relevant job names / previous jobs and organisations
'''
def getJobsInfo(filename):
    text = getData(filename)
    #text = text
    text = text.replace('●','').replace('','')
    gg = extract_entity_sections_professional2(text)
    gg1 = extract_entity_sections_professional(text)
    temp_list2 = []
    temp_list3 = []
    averageChange = 0
    last_item = 0
    for i in gg:
        if len(i) < 2:
            pos1 = gg.index(i)
            del gg[pos1]
        match = re.search("[0-9]{4}\s",i.strip().lower())
        if match:
            temp_list = []
            pos = gg.index(i)
            temp_list.append(i)
            #check if the item after the date is longer than the item before
            if pos == len(gg)-1:
                temp_list2.insert(0,gg[0])
                last_item = 1
                eachDate = get_total_experience(temp_list)
                temp_list3.append(eachDate)
                break
            if last_item == 0:
                y = re.search("\,\s",i)
                if y:
                    y_check = True
    #            if len(gg[pos-1]) > len(gg[pos+1]):  
    #                temp_list2.append(gg[pos+1])
    #            elif len(gg[pos+1]) > len(gg[pos-1]):
    #                temp_list2.append(gg[pos-1])
    #            if 
                
                temp_list2.append(gg[pos-1])
#            print(temp_list)
            eachDate = get_total_experience(temp_list)
#            print(eachDate)
            temp_list3.append(eachDate)
    #    changeRate = []
    if len(temp_list3) == 1:
            averageChange = temp_list3[0]
    if len(temp_list3) >= 2:
        averageChange = sum(temp_list3)/len(temp_list3)

    return temp_list2,temp_list3,averageChange,gg1

In [18]:
'''
Function to get all the information from all functions into a dataframe
'''
def resume_object(filename): 
    
    data1 = getData(filename)
    #name Extract
    res_name = final_names(data1)
    #Extract professional2
    entities2 = extract_entity_sections_professional2(data1)
    #Extract No of Experience
    experience = get_total_experience(entities2)
    experience = round(experience/12,2)
    #All phone number, education and email
    rest_detail = alldetails(data1)
    #Name of File
    filename_p = os.path.basename(filename)
    #Job Related Details
    jobDetail = getJobsInfo(filename)
    #Experience in words
    experience_word = extract_entity_sections_professional2(data1.replace("â€œ",'').replace("”",'').replace("“",'').replace("–",'').replace("●",'').replace("⚫",'').replace("❖",'').replace("\uf0a2",'').replace("•",'').replace("—",'').replace("’",''))
    experience_word = ", ".join(experience_word)
    #Wordcloud text
    word_cloud = wordcloud(data1)
    rest_detail['Name'] = res_name
    rest_detail['Total Experience in years'] = experience
    rest_detail['Filename'] = filename_p
    rest_detail['Experience'] = experience_word
    rest_detail['Word Cloud'] = word_cloud
    job_role = jobDetail[0]
    rest_detail['Job Role'] = job_role
    rest_detail['Duration Per Job(m)'] = jobDetail[1]
#    rest_detail[''] = jobDetail[]
    rest_detail['Average Change'] = jobDetail[2]
    rest_detail['Job Desc'] = jobDetail[3]['experience']
    rest_detail['Edu Desc'] = jobDetail[3]['education']
    
    #All Skills in dictionary depend on number of skill files
    allskills = skills_extraction(data1)
    sp = skills_point(allskills)
    rest_detail['dE_no'] = sp[0]
    rest_detail['gth_no'] = sp[1]
    rest_detail['ra_no'] = sp[2]
    rest_detail['ld_no'] = sp[3]
    rest_detail['rp_no'] = sp[4]
    
    resume_obj= {**rest_detail, **allskills}
    return resume_obj

'''
get the name of the pdf 
'''
def printfiles(directory):
    listofresume = []
    for filename in os.listdir(directory):
        if filename.endswith('.pdf'):
            listofresume.append(os.path.join(directory, filename))
            # print(os.path.join(pathoffolder, filename))
            continue
        else:
            continue
    return listofresume

In [19]:
'''
Converting the dataframe into a csv to store
'''
def dict_to_csv():
    csv_columns = ['Filename','Name','Phone Number','Email Address','Job Role','Duration Per Job(m)','Average Change','Job Desc','Edu Desc','Total Experience in years','education','good_to_have','leadership','dataEn_skill','relevant_ability','results_performance','Experience','Word Cloud','total_skills','dE_no','gth_no','ra_no','ld_no','rp_no','Education Level']
    df = pd.DataFrame(columns = csv_columns)
    directory = "C:/Users/TP_baseline/Documents/major_project/Resume"
    for x in printfiles(directory):
        print("Processing {} Resume Now..".format(x))
        t = time.time()
        resume_de = resume_object(x)
        df = df.append(resume_de,ignore_index=True)
        print("time taken:%.2f" % (time.time()-t))
    df.to_csv(r'C:/Users/TP_baseline/Documents/major_project/resume_data.csv', index=False)
    print("Conversion Completed")
    return df

In [20]:
'''
Calling the function and getting the estimated time
'''
t = time.time()
gotit = dict_to_csv()
print("\ntime taken:%.2f" % (time.time()-t))

Processing C:/Users/TP_baseline/Documents/major_project/Resume\Javier.pdf Resume Now..
time taken:2.95
Processing C:/Users/TP_baseline/Documents/major_project/Resume\JiaXin.pdf Resume Now..
time taken:1.83
Processing C:/Users/TP_baseline/Documents/major_project/Resume\Jonathan.pdf Resume Now..
time taken:1.07
Processing C:/Users/TP_baseline/Documents/major_project/Resume\LingPing.pdf Resume Now..
time taken:1.06
Processing C:/Users/TP_baseline/Documents/major_project/Resume\Patricia.pdf Resume Now..
time taken:1.01
Conversion Completed

time taken:7.98


In [21]:
gotit

,Filename,Name,Phone Number,Email Address,Job Role,Duration Per Job(m),Average Change,Job Desc,Edu Desc,Total Experience in years,...,results_performance,Experience,Word Cloud,total_skills,dE_no,gth_no,ra_no,ld_no,rp_no,Education Level
0,Javier.pdf,jav,+65-65923456,javier456@hotmail.com,[(sneaker hotspot news series) to promote the ...,"[18, 31]",24.500000,"[process integration engineer, micron, singapo...","[master of science in eee, gpa: 4/5 ...",4.08,...,"pinpoint, improve, accuracy, enhanced, drive, ...","process integration engineer, micron, singapor...",javier tel 6592345678 email javier456 hotma...,"coding, root cause, jupyter, jmp, certificatio...",6,2,1,3,6,ntu
1,JiaXin.pdf,qiujiaxinobjectiv,+65-65829230,jiaxinqiu@gmail.com,[responsible for image processing and algorith...,"[9, 1]",5.000000,[project 5c: post quantum security as a servic...,[master of computer science (gpa 3.65 / 5.0) ...,1.58,...,"improve, drive, greatly",project 5c: post quantum security as a service...,comprehensive ability nus master study experi...,"sales, video, powerpoint, word, xml, electroni...",9,1,2,4,3,nus
2,Jonathan.pdf,jonathanlim,+65-65824297,jonathan@gmail.com,"[ace pointer full stack developer, api documen...","[3, 41, 12]",18.666667,"[ace pointer, full stack developer, deployment...","[national university of singapore, master of s...",4.67,...,Nan,"ace pointer, full stack developer, deployment ...",jonathan lim contact information email jonatha...,"electrical engineering, electronics, django, i...",3,1,1,1,1,nus
3,LingPing.pdf,tanlingpingping,+65-96240378,tanlingping@gmail.com,"[data scientist intern, agoda, data scientist ...","[8, 8, 2, 2, 4]",4.800000,"[data scientist intern, agoda, sep 2018 — may ...","[nanyang technological university, singapore, ...",2.00,...,predict,"data scientist intern, agoda, sep 2018 may 20...",tan ling ping mobile 96240378 email tanling...,"statistics, research, python, analysis, tensor...",6,1,2,1,1,ntu
4,Patricia.pdf,patriciachen,+65-89241058,patrciachen@gmail.com,"[airliquide.com, singapore - aws cloud system...","[9, 7, 12]",9.333333,"[airliquide.com, singapore - aws cloud system...","[pamantasan lungsod ng maynila, philippines - ...",2.33,...,Nan,"airliquide.com, singapore - aws cloud system ...",patricia chen devops engineer blk 461yishun a...,"wordpress, vmware, certification, shell, aws c...",4,3,1,1,1,Nan


## Rule Based Engine

In [22]:
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

In [23]:
pd.set_option('display.max_colwidth', 20)

In [24]:
'''
Read the csv
'''
loldf = pd.read_csv("resume_data.csv")

gg1 = loldf
incase = loldf

In [25]:
'''
Set the filename as index
'''
gg1 = gg1.set_index('Filename')
loldf = loldf.set_index('Filename')

In [26]:
gg1.drop(columns=['Phone Number', 'Email Address','Word Cloud','total_skills'])

,Name,Job Role,Duration Per Job(m),Average Change,Job Desc,Edu Desc,Total Experience in years,education,good_to_have,leadership,dataEn_skill,relevant_ability,results_performance,Experience,dE_no,gth_no,ra_no,ld_no,rp_no,Education Level
Filename,,,,,,,,,,,,,,,,,,,,
Javier.pdf,jav,['(sneaker hotsp...,"[18, 31]",24.500000,['process integr...,['master of scie...,4.08,master,"numpy, machine l...","cooperate, overs...","python, sql",Nan,"pinpoint, improv...",process integrat...,6,2,1,3,6,ntu
JiaXin.pdf,qiujiaxinobjectiv,['responsible fo...,"[9, 1]",5.000000,['project 5c: po...,['master of comp...,1.58,"master, bachelor",image processing,"responsible, tea...","python, sql, java","model, data anal...","improve, drive, ...",project 5c: post...,9,1,2,4,3,nus
Jonathan.pdf,jonathanlim,['ace pointer fu...,"[3, 41, 12]",18.666667,"['ace pointer', ...",['national unive...,4.67,"master, bachelor",aws,proposed,python,Nan,Nan,"ace pointer, ful...",3,1,1,1,1,nus
LingPing.pdf,tanlingpingping,['data scientist...,"[8, 8, 2, 2, 4]",4.800000,['data scientist...,['nanyang techno...,2.00,bachelor,Nan,Nan,"python, sql",feature preparat...,predict,data scientist i...,6,1,2,1,1,ntu
Patricia.pdf,patriciachen,['airliquide.com...,"[9, 7, 12]",9.333333,['airliquide.com...,['pamantasan lun...,2.33,Nan,"docker, shell, aws",spearheaded,"python, linux",Nan,Nan,"airliquide.com, ...",4,3,1,1,1,Nan


In [27]:
gg1['Job Applied'] = ['Data Engineer','Data Engineer','Software Developer','Data Engineer','Dev Ops']

In [28]:
'''
Sort by Data Engineer and began criteria checking
'''
gg = gg1[gg1['Job Applied'] == 'Data Engineer']

In [29]:
gg

,Name,Phone Number,Email Address,Job Role,Duration Per Job(m),Average Change,Job Desc,Edu Desc,Total Experience in years,education,good_to_have,leadership,dataEn_skill,relevant_ability,results_performance,Experience,Word Cloud,total_skills,dE_no,gth_no,ra_no,ld_no,rp_no,Education Level,Job Applied
Filename,,,,,,,,,,,,,,,,,,,,,,,,,
Javier.pdf,jav,+65-65923456,javier456@hotmai...,['(sneaker hotsp...,"[18, 31]",24.5,['process integr...,['master of scie...,4.08,master,"numpy, machine l...","cooperate, overs...","python, sql",Nan,"pinpoint, improv...",process integrat...,javier tel 6592...,"coding, root cau...",6,2,1,3,6,ntu,Data Engineer
JiaXin.pdf,qiujiaxinobjectiv,+65-65829230,jiaxinqiu@gmail.com,['responsible fo...,"[9, 1]",5.0,['project 5c: po...,['master of comp...,1.58,"master, bachelor",image processing,"responsible, tea...","python, sql, java","model, data anal...","improve, drive, ...",project 5c: post...,comprehensive ab...,"sales, video, po...",9,1,2,4,3,nus,Data Engineer
LingPing.pdf,tanlingpingping,+65-96240378,tanlingping@gmai...,['data scientist...,"[8, 8, 2, 2, 4]",4.8,['data scientist...,['nanyang techno...,2.00,bachelor,Nan,Nan,"python, sql",feature preparat...,predict,data scientist i...,tan ling ping mo...,"statistics, rese...",6,1,2,1,1,ntu,Data Engineer


In [30]:
resultdf = pd.DataFrame()

In [31]:
resultdf['Filename'] = gg.index

In [32]:
resultdf = resultdf.set_index('Filename')

In [33]:
'''
45% for data engineer skills include example python = 3p,
'''
def dataEngineer_no(gg):
    new1 = gg['dE_no'].tolist()
    Lst = []
    for i in new1:
        per = round(((i/14) * 45.0),3)
        Lst.append(per)
    return Lst

resultdf['dataEngSkill'] = dataEngineer_no(gg)
dataEngineer_no((gg))

[19.286, 28.929, 19.286]

In [34]:
'''
Good to have skills 5%
'''
def goodtoHave_no(gg):
    new1 = gg['gth_no'].tolist()
    dE_noLst = []
    for i in new1:
        per = round(((i/12) * 5.0),3)
        dE_noLst.append(per)
    return dE_noLst

resultdf['gth_skills'] = goodtoHave_no(gg)
goodtoHave_no(gg)

[0.833, 0.417, 0.417]

In [35]:
'''
Leadership skills for 2%
'''
def leadership_no(gg):
    new1 = gg['ld_no'].tolist()
    Lst = []
    for i in new1:
        per = round(((i/10) * 2.0),3)
        Lst.append(per)
    return Lst

resultdf['leader'] = leadership_no(gg)

In [36]:
'''
Relevant Ability 11%
'''
def relevant_abil_no(gg):
    new1 = gg['ra_no'].tolist()
    Lst = []
    for i in new1:
        per = round(((i/17) * 11.0),3)
        Lst.append(per)
    return Lst

resultdf['relevant_skill'] = relevant_abil_no(gg)

In [37]:
'''
Performance keywords 2%
'''
def perform_no(gg):
    new1 = gg['ra_no'].tolist()
    Lst = []
    for i in new1:
        per = round(((i/17) * 2.0),3)
        Lst.append(per)
    return Lst

resultdf['p_lvl'] = perform_no(gg)

In [38]:
'''
has relevant_jobs 5%
'''
def relevant_job(gg):
    new1 = gg['Job Desc'].tolist()
    Lst = []
    for i in new1:
        i = re.sub(r'\W+', '', i).lower()
#         print(i)
    for i in new1:
        if "data engineer" in i or "data scientist" in i or "analyst" in i:
            Lst.append(5.0)
        else:
            Lst.append(0.0)
    return Lst

resultdf['rele_job'] = relevant_job(gg)

In [39]:
#NNEEDD Level + Degree
'''
Retrieve specialisation degree example bachelor of science in computing
'''
def specialise(gg):
    new1 = gg['Edu Desc'].tolist()
    Lst = []
    finLst = []
    tempLst2 = []
    for i in new1:
        i = i.strip().replace('[','').replace(']','')
        iLst = i.split("', ")
        tmpLst = []
        for g in iLst:
            if 'master' in g or 'b.' in g or 'bachelor' in g or 'b.s.' in g:
                g = g.replace("'",'')
                tmpLst.append(g)
        finLst.append(tmpLst)
        print(tmpLst)
        print('\n')
    for a in finLst:
        tempLst = []
        tempLst1 = []
        specLst = []
        for item in a:
            if 'master' in item and 'computer' in item:
                tempLst.append(20.0)
                specLst.append(item)
            elif 'diploma' in item and 'software' in item:
                tempLst.append(20.0)
                specLst.append(item)
            elif 'b.s' in item and 'comput' in item:
                tempLst.append(20.0)
                specLst.append(item)
            elif 'bachelor' in item and 'comput' in item:
                tempLst.append(20.0)
                specLst.append(item)
            elif 'b.eng' in item:
                tempLst.append(20.0)
                specLst.append(item)
            elif 'b.' in item:
                tempLst.append(20.0)
                specLst.append(item)
            elif 'bachelor of engineer' in item:
                tempLst.append(20.0)
                specLst.append(item)
            else:
                tempLst.append(0.0)
                specLst.append('Not Relevant')
        if len(tempLst) > 1:
            result = max(tempLst)
        else:
            result = tempLst[0]
        Lst.append(result)
        print(specLst)
        if len(specLst) > 1:
            for i in specLst:
                if 'Not Relevant' not in i:
                    tempLst2.append(i)
            if len(tempLst2) == 0:
                tempLst.append('Not Relevant')
        else:
            if 'Not Relevant' not in specLst:
                print(specLst)
                tempLst2.append(specLst[0])
            else:
                tempLst2.append('Not Relevant')
    
    return Lst,tempLst2

resultdf['specialise_no'] = specialise(gg)[0]
gg['specialise'] = specialise(gg)[1]

['master of science in eee, gpa: 4/5                                             aug 2017 - jul 2018 nanyang technological', 'b.eng. in mechanical design, manufacturing and automation, gpa: 3.78/4           sep 2013 - jun 2017']


['master of computer science (gpa 3.65 / 5.0)           national university of singapore                aug 2018-jul 2019', 'bachelor of information safety (gpa 3.83 / 4.0)']


['bachelor of science in applied physics']


['Not Relevant', 'b.eng. in mechanical design, manufacturing and automation, gpa: 3.78/4           sep 2013 - jun 2017']
['master of computer science (gpa 3.65 / 5.0)           national university of singapore                aug 2018-jul 2019', 'Not Relevant']
['Not Relevant']
['master of science in eee, gpa: 4/5                                             aug 2017 - jul 2018 nanyang technological', 'b.eng. in mechanical design, manufacturing and automation, gpa: 3.78/4           sep 2013 - jun 2017']


['master of computer science (gpa 3.65

In [40]:
# gg['specialise'].values

In [41]:
'''
Check if Total Experience in years if more than 1 5%
'''

def experience_no(gg):
    new1 = gg['Total Experience in years'].tolist()
    Lst = []
    for i in new1:
        if i >= 1:
            Lst.append(5.0)
        else:
            Lst.append(0.0)
    return Lst

resultdf['exp_no'] = experience_no(gg)

In [42]:
'''
Check if university is local like nus, ntu,smu 5%
'''
def edu1(gg):
    new1 = gg['Education Level'].tolist()
    Lst = []
    for i in new1:
        i = re.sub(r'\W+', '', i).lower()
#         print(i)
    for i in new1:
        if "ntu" in i or "nus" in i or "smu" in i:
            Lst.append(5.0)
        else:
            Lst.append(0.0)
    return Lst

resultdf['sg_sch'] = edu1(gg)

In [43]:
resultdf['total_per'] = resultdf.sum(axis=1)
resultdf

,dataEngSkill,gth_skills,leader,relevant_skill,p_lvl,rele_job,specialise_no,exp_no,sg_sch,total_per
Filename,,,,,,,,,,
Javier.pdf,19.286,0.833,0.6,0.647,0.118,5.0,20.0,5.0,5.0,56.484
JiaXin.pdf,28.929,0.417,0.8,1.294,0.235,0.0,20.0,5.0,5.0,61.675
LingPing.pdf,19.286,0.417,0.2,1.294,0.235,5.0,0.0,5.0,5.0,36.432


In [44]:
gg

,Name,Phone Number,Email Address,Job Role,Duration Per Job(m),Average Change,Job Desc,Edu Desc,Total Experience in years,education,good_to_have,leadership,dataEn_skill,relevant_ability,results_performance,Experience,Word Cloud,total_skills,dE_no,gth_no,ra_no,ld_no,rp_no,Education Level,Job Applied,specialise
Filename,,,,,,,,,,,,,,,,,,,,,,,,,,
Javier.pdf,jav,+65-65923456,javier456@hotmai...,['(sneaker hotsp...,"[18, 31]",24.5,['process integr...,['master of scie...,4.08,master,"numpy, machine l...","cooperate, overs...","python, sql",Nan,"pinpoint, improv...",process integrat...,javier tel 6592...,"coding, root cau...",6,2,1,3,6,ntu,Data Engineer,b.eng. in mechan...
JiaXin.pdf,qiujiaxinobjectiv,+65-65829230,jiaxinqiu@gmail.com,['responsible fo...,"[9, 1]",5.0,['project 5c: po...,['master of comp...,1.58,"master, bachelor",image processing,"responsible, tea...","python, sql, java","model, data anal...","improve, drive, ...",project 5c: post...,comprehensive ab...,"sales, video, po...",9,1,2,4,3,nus,Data Engineer,master of comput...
LingPing.pdf,tanlingpingping,+65-96240378,tanlingping@gmai...,['data scientist...,"[8, 8, 2, 2, 4]",4.8,['data scientist...,['nanyang techno...,2.00,bachelor,Nan,Nan,"python, sql",feature preparat...,predict,data scientist i...,tan ling ping mo...,"statistics, rese...",6,1,2,1,1,ntu,Data Engineer,Not Relevant


In [45]:
final_df = pd.concat([gg,resultdf], axis=1)

In [46]:
final_df

,Name,Phone Number,Email Address,Job Role,Duration Per Job(m),Average Change,Job Desc,Edu Desc,Total Experience in years,education,good_to_have,leadership,dataEn_skill,relevant_ability,results_performance,Experience,Word Cloud,total_skills,dE_no,gth_no,ra_no,ld_no,rp_no,Education Level,Job Applied,specialise,dataEngSkill,gth_skills,leader,relevant_skill,p_lvl,rele_job,specialise_no,exp_no,sg_sch,total_per
Filename,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Javier.pdf,jav,+65-65923456,javier456@hotmai...,['(sneaker hotsp...,"[18, 31]",24.5,['process integr...,['master of scie...,4.08,master,"numpy, machine l...","cooperate, overs...","python, sql",Nan,"pinpoint, improv...",process integrat...,javier tel 6592...,"coding, root cau...",6,2,1,3,6,ntu,Data Engineer,b.eng. in mechan...,19.286,0.833,0.6,0.647,0.118,5.0,20.0,5.0,5.0,56.484
JiaXin.pdf,qiujiaxinobjectiv,+65-65829230,jiaxinqiu@gmail.com,['responsible fo...,"[9, 1]",5.0,['project 5c: po...,['master of comp...,1.58,"master, bachelor",image processing,"responsible, tea...","python, sql, java","model, data anal...","improve, drive, ...",project 5c: post...,comprehensive ab...,"sales, video, po...",9,1,2,4,3,nus,Data Engineer,master of comput...,28.929,0.417,0.8,1.294,0.235,0.0,20.0,5.0,5.0,61.675
LingPing.pdf,tanlingpingping,+65-96240378,tanlingping@gmai...,['data scientist...,"[8, 8, 2, 2, 4]",4.8,['data scientist...,['nanyang techno...,2.00,bachelor,Nan,Nan,"python, sql",feature preparat...,predict,data scientist i...,tan ling ping mo...,"statistics, rese...",6,1,2,1,1,ntu,Data Engineer,Not Relevant,19.286,0.417,0.2,1.294,0.235,5.0,0.0,5.0,5.0,36.432


In [47]:
do = gg1[gg1['Job Applied'] == 'Software Developer']
do['total_per'] = 25.0

In [48]:
sd = gg1[gg1['Job Applied'] == 'Dev Ops']
sd['total_per'] = 25.0

In [49]:
final_df90 = pd.concat([final_df,sd,do])
final_df90.head()

,Average Change,Duration Per Job(m),Edu Desc,Education Level,Email Address,Experience,Job Applied,Job Desc,Job Role,Name,Phone Number,Total Experience in years,Word Cloud,dE_no,dataEn_skill,dataEngSkill,education,exp_no,good_to_have,gth_no,gth_skills,ld_no,leader,leadership,p_lvl,ra_no,rele_job,relevant_ability,relevant_skill,results_performance,rp_no,sg_sch,specialise,specialise_no,total_per,total_skills
Filename,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Javier.pdf,24.500000,"[18, 31]",['master of scie...,ntu,javier456@hotmai...,process integrat...,Data Engineer,['process integr...,['(sneaker hotsp...,jav,+65-65923456,4.08,javier tel 6592...,6,"python, sql",19.286,master,5.0,"numpy, machine l...",2,0.833,3,0.6,"cooperate, overs...",0.118,1,5.0,Nan,0.647,"pinpoint, improv...",6,5.0,b.eng. in mechan...,20.0,56.484,"coding, root cau..."
JiaXin.pdf,5.000000,"[9, 1]",['master of comp...,nus,jiaxinqiu@gmail.com,project 5c: post...,Data Engineer,['project 5c: po...,['responsible fo...,qiujiaxinobjectiv,+65-65829230,1.58,comprehensive ab...,9,"python, sql, java",28.929,"master, bachelor",5.0,image processing,1,0.417,4,0.8,"responsible, tea...",0.235,2,0.0,"model, data anal...",1.294,"improve, drive, ...",3,5.0,master of comput...,20.0,61.675,"sales, video, po..."
LingPing.pdf,4.800000,"[8, 8, 2, 2, 4]",['nanyang techno...,ntu,tanlingping@gmai...,data scientist i...,Data Engineer,['data scientist...,['data scientist...,tanlingpingping,+65-96240378,2.00,tan ling ping mo...,6,"python, sql",19.286,bachelor,5.0,Nan,1,0.417,1,0.2,Nan,0.235,2,5.0,feature preparat...,1.294,predict,1,5.0,Not Relevant,0.0,36.432,"statistics, rese..."
Patricia.pdf,9.333333,"[9, 7, 12]",['pamantasan lun...,Nan,patrciachen@gmai...,"airliquide.com, ...",Dev Ops,['airliquide.com...,['airliquide.com...,patriciachen,+65-89241058,2.33,patricia chen de...,4,"python, linux",NaN,Nan,NaN,"docker, shell, aws",3,NaN,1,NaN,spearheaded,NaN,1,NaN,Nan,NaN,Nan,1,NaN,NaN,NaN,25.000,"wordpress, vmwar..."
Jonathan.pdf,18.666667,"[3, 41, 12]",['national unive...,nus,jonathan@gmail.com,"ace pointer, ful...",Software Developer,"['ace pointer', ...",['ace pointer fu...,jonathanlim,+65-65824297,4.67,jonathan lim con...,3,python,NaN,"master, bachelor",NaN,aws,1,NaN,1,NaN,proposed,NaN,1,NaN,Nan,NaN,Nan,1,NaN,NaN,NaN,25.000,electrical engin...


In [50]:
'''
Sort by order
'''
final_df90.sort_values(by= 'total_per',ascending = False,inplace=True)

In [51]:
'''
Save as csv
'''
final_df90.to_csv('final.csv',index=True)

In [52]:
final_df90

,Average Change,Duration Per Job(m),Edu Desc,Education Level,Email Address,Experience,Job Applied,Job Desc,Job Role,Name,Phone Number,Total Experience in years,Word Cloud,dE_no,dataEn_skill,dataEngSkill,education,exp_no,good_to_have,gth_no,gth_skills,ld_no,leader,leadership,p_lvl,ra_no,rele_job,relevant_ability,relevant_skill,results_performance,rp_no,sg_sch,specialise,specialise_no,total_per,total_skills
Filename,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
JiaXin.pdf,5.000000,"[9, 1]",['master of comp...,nus,jiaxinqiu@gmail.com,project 5c: post...,Data Engineer,['project 5c: po...,['responsible fo...,qiujiaxinobjectiv,+65-65829230,1.58,comprehensive ab...,9,"python, sql, java",28.929,"master, bachelor",5.0,image processing,1,0.417,4,0.8,"responsible, tea...",0.235,2,0.0,"model, data anal...",1.294,"improve, drive, ...",3,5.0,master of comput...,20.0,61.675,"sales, video, po..."
Javier.pdf,24.500000,"[18, 31]",['master of scie...,ntu,javier456@hotmai...,process integrat...,Data Engineer,['process integr...,['(sneaker hotsp...,jav,+65-65923456,4.08,javier tel 6592...,6,"python, sql",19.286,master,5.0,"numpy, machine l...",2,0.833,3,0.6,"cooperate, overs...",0.118,1,5.0,Nan,0.647,"pinpoint, improv...",6,5.0,b.eng. in mechan...,20.0,56.484,"coding, root cau..."
LingPing.pdf,4.800000,"[8, 8, 2, 2, 4]",['nanyang techno...,ntu,tanlingping@gmai...,data scientist i...,Data Engineer,['data scientist...,['data scientist...,tanlingpingping,+65-96240378,2.00,tan ling ping mo...,6,"python, sql",19.286,bachelor,5.0,Nan,1,0.417,1,0.2,Nan,0.235,2,5.0,feature preparat...,1.294,predict,1,5.0,Not Relevant,0.0,36.432,"statistics, rese..."
Patricia.pdf,9.333333,"[9, 7, 12]",['pamantasan lun...,Nan,patrciachen@gmai...,"airliquide.com, ...",Dev Ops,['airliquide.com...,['airliquide.com...,patriciachen,+65-89241058,2.33,patricia chen de...,4,"python, linux",NaN,Nan,NaN,"docker, shell, aws",3,NaN,1,NaN,spearheaded,NaN,1,NaN,Nan,NaN,Nan,1,NaN,NaN,NaN,25.000,"wordpress, vmwar..."
Jonathan.pdf,18.666667,"[3, 41, 12]",['national unive...,nus,jonathan@gmail.com,"ace pointer, ful...",Software Developer,"['ace pointer', ...",['ace pointer fu...,jonathanlim,+65-65824297,4.67,jonathan lim con...,3,python,NaN,"master, bachelor",NaN,aws,1,NaN,1,NaN,proposed,NaN,1,NaN,Nan,NaN,Nan,1,NaN,NaN,NaN,25.000,electrical engin...


In [53]:
final_df90.columns

Index(['Average Change', 'Duration Per Job(m)', 'Edu Desc', 'Education Level',
       'Email Address', 'Experience', 'Job Applied', 'Job Desc', 'Job Role',
       'Name', 'Phone Number', 'Total Experience in years', 'Word Cloud',
       'dE_no', 'dataEn_skill', 'dataEngSkill', 'education', 'exp_no',
       'good_to_have', 'gth_no', 'gth_skills', 'ld_no', 'leader', 'leadership',
       'p_lvl', 'ra_no', 'rele_job', 'relevant_ability', 'relevant_skill',
       'results_performance', 'rp_no', 'sg_sch', 'specialise', 'specialise_no',
       'total_per', 'total_skills'],
      dtype='object')

In [54]:
newFinal = pd.read_csv("final.csv")

In [55]:
newFinal

,Filename,Average Change,Duration Per Job(m),Edu Desc,Education Level,Email Address,Experience,Job Applied,Job Desc,Job Role,Name,Phone Number,Total Experience in years,Word Cloud,dE_no,dataEn_skill,dataEngSkill,education,exp_no,good_to_have,gth_no,gth_skills,ld_no,leader,leadership,p_lvl,ra_no,rele_job,relevant_ability,relevant_skill,results_performance,rp_no,sg_sch,specialise,specialise_no,total_per,total_skills
0,JiaXin.pdf,5.000000,"[9, 1]",['master of comp...,nus,jiaxinqiu@gmail.com,project 5c: post...,Data Engineer,['project 5c: po...,['responsible fo...,qiujiaxinobjectiv,+65-65829230,1.58,comprehensive ab...,9,"python, sql, java",28.929,"master, bachelor",5.0,image processing,1,0.417,4,0.8,"responsible, tea...",0.235,2,0.0,"model, data anal...",1.294,"improve, drive, ...",3,5.0,master of comput...,20.0,61.675,"sales, video, po..."
1,Javier.pdf,24.500000,"[18, 31]",['master of scie...,ntu,javier456@hotmai...,process integrat...,Data Engineer,['process integr...,['(sneaker hotsp...,jav,+65-65923456,4.08,javier tel 6592...,6,"python, sql",19.286,master,5.0,"numpy, machine l...",2,0.833,3,0.6,"cooperate, overs...",0.118,1,5.0,Nan,0.647,"pinpoint, improv...",6,5.0,b.eng. in mechan...,20.0,56.484,"coding, root cau..."
2,LingPing.pdf,4.800000,"[8, 8, 2, 2, 4]",['nanyang techno...,ntu,tanlingping@gmai...,data scientist i...,Data Engineer,['data scientist...,['data scientist...,tanlingpingping,+65-96240378,2.00,tan ling ping mo...,6,"python, sql",19.286,bachelor,5.0,Nan,1,0.417,1,0.2,Nan,0.235,2,5.0,feature preparat...,1.294,predict,1,5.0,Not Relevant,0.0,36.432,"statistics, rese..."
3,Patricia.pdf,9.333333,"[9, 7, 12]",['pamantasan lun...,Nan,patrciachen@gmai...,"airliquide.com, ...",Dev Ops,['airliquide.com...,['airliquide.com...,patriciachen,+65-89241058,2.33,patricia chen de...,4,"python, linux",NaN,Nan,NaN,"docker, shell, aws",3,NaN,1,NaN,spearheaded,NaN,1,NaN,Nan,NaN,Nan,1,NaN,NaN,NaN,25.000,"wordpress, vmwar..."
4,Jonathan.pdf,18.666667,"[3, 41, 12]",['national unive...,nus,jonathan@gmail.com,"ace pointer, ful...",Software Developer,"['ace pointer', ...",['ace pointer fu...,jonathanlim,+65-65824297,4.67,jonathan lim con...,3,python,NaN,"master, bachelor",NaN,aws,1,NaN,1,NaN,proposed,NaN,1,NaN,Nan,NaN,Nan,1,NaN,NaN,NaN,25.000,electrical engin...


In [56]:
newFinal

,Filename,Average Change,Duration Per Job(m),Edu Desc,Education Level,Email Address,Experience,Job Applied,Job Desc,Job Role,Name,Phone Number,Total Experience in years,Word Cloud,dE_no,dataEn_skill,dataEngSkill,education,exp_no,good_to_have,gth_no,gth_skills,ld_no,leader,leadership,p_lvl,ra_no,rele_job,relevant_ability,relevant_skill,results_performance,rp_no,sg_sch,specialise,specialise_no,total_per,total_skills
0,JiaXin.pdf,5.000000,"[9, 1]",['master of comp...,nus,jiaxinqiu@gmail.com,project 5c: post...,Data Engineer,['project 5c: po...,['responsible fo...,qiujiaxinobjectiv,+65-65829230,1.58,comprehensive ab...,9,"python, sql, java",28.929,"master, bachelor",5.0,image processing,1,0.417,4,0.8,"responsible, tea...",0.235,2,0.0,"model, data anal...",1.294,"improve, drive, ...",3,5.0,master of comput...,20.0,61.675,"sales, video, po..."
1,Javier.pdf,24.500000,"[18, 31]",['master of scie...,ntu,javier456@hotmai...,process integrat...,Data Engineer,['process integr...,['(sneaker hotsp...,jav,+65-65923456,4.08,javier tel 6592...,6,"python, sql",19.286,master,5.0,"numpy, machine l...",2,0.833,3,0.6,"cooperate, overs...",0.118,1,5.0,Nan,0.647,"pinpoint, improv...",6,5.0,b.eng. in mechan...,20.0,56.484,"coding, root cau..."
2,LingPing.pdf,4.800000,"[8, 8, 2, 2, 4]",['nanyang techno...,ntu,tanlingping@gmai...,data scientist i...,Data Engineer,['data scientist...,['data scientist...,tanlingpingping,+65-96240378,2.00,tan ling ping mo...,6,"python, sql",19.286,bachelor,5.0,Nan,1,0.417,1,0.2,Nan,0.235,2,5.0,feature preparat...,1.294,predict,1,5.0,Not Relevant,0.0,36.432,"statistics, rese..."
3,Patricia.pdf,9.333333,"[9, 7, 12]",['pamantasan lun...,Nan,patrciachen@gmai...,"airliquide.com, ...",Dev Ops,['airliquide.com...,['airliquide.com...,patriciachen,+65-89241058,2.33,patricia chen de...,4,"python, linux",NaN,Nan,NaN,"docker, shell, aws",3,NaN,1,NaN,spearheaded,NaN,1,NaN,Nan,NaN,Nan,1,NaN,NaN,NaN,25.000,"wordpress, vmwar..."
4,Jonathan.pdf,18.666667,"[3, 41, 12]",['national unive...,nus,jonathan@gmail.com,"ace pointer, ful...",Software Developer,"['ace pointer', ...",['ace pointer fu...,jonathanlim,+65-65824297,4.67,jonathan lim con...,3,python,NaN,"master, bachelor",NaN,aws,1,NaN,1,NaN,proposed,NaN,1,NaN,Nan,NaN,Nan,1,NaN,NaN,NaN,25.000,electrical engin...


In [57]:
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

In [58]:
newFinal = newFinal.applymap(str)
newFinal = newFinal.apply(lambda x: x.str.strip())
newFinal.applymap(_removeNonAscii)

,Filename,Average Change,Duration Per Job(m),Edu Desc,Education Level,Email Address,Experience,Job Applied,Job Desc,Job Role,Name,Phone Number,Total Experience in years,Word Cloud,dE_no,dataEn_skill,dataEngSkill,education,exp_no,good_to_have,gth_no,gth_skills,ld_no,leader,leadership,p_lvl,ra_no,rele_job,relevant_ability,relevant_skill,results_performance,rp_no,sg_sch,specialise,specialise_no,total_per,total_skills
0,JiaXin.pdf,5.0,"[9, 1]",['master of comp...,nus,jiaxinqiu@gmail.com,project 5c: post...,Data Engineer,['project 5c: po...,['responsible fo...,qiujiaxinobjectiv,+65-65829230,1.58,comprehensive ab...,9,"python, sql, java",28.929000000000002,"master, bachelor",5.0,image processing,1,0.41700000000000004,4,0.8,"responsible, tea...",0.235,2,0.0,"model, data anal...",1.294,"improve, drive, ...",3,5.0,master of comput...,20.0,61.675,"sales, video, po..."
1,Javier.pdf,24.5,"[18, 31]",['master of scie...,ntu,javier456@hotmai...,process integrat...,Data Engineer,['process integr...,['(sneaker hotsp...,jav,+65-65923456,4.08,javier tel 6592...,6,"python, sql",19.285999999999998,master,5.0,"numpy, machine l...",2,0.833,3,0.6,"cooperate, overs...",0.11800000000000001,1,5.0,Nan,0.647,"pinpoint, improv...",6,5.0,b.eng. in mechan...,20.0,56.483999999999995,"coding, root cau..."
2,LingPing.pdf,4.8,"[8, 8, 2, 2, 4]",['nanyang techno...,ntu,tanlingping@gmai...,data scientist i...,Data Engineer,['data scientist...,['data scientist...,tanlingpingping,+65-96240378,2.0,tan ling ping mo...,6,"python, sql",19.285999999999998,bachelor,5.0,Nan,1,0.41700000000000004,1,0.2,Nan,0.235,2,5.0,feature preparat...,1.294,predict,1,5.0,Not Relevant,0.0,36.431999999999995,"statistics, rese..."
3,Patricia.pdf,9.333333333333334,"[9, 7, 12]",['pamantasan lun...,Nan,patrciachen@gmai...,"airliquide.com, ...",Dev Ops,['airliquide.com...,['airliquide.com...,patriciachen,+65-89241058,2.33,patricia chen de...,4,"python, linux",nan,Nan,nan,"docker, shell, aws",3,nan,1,nan,spearheaded,nan,1,nan,Nan,nan,Nan,1,nan,nan,nan,25.0,"wordpress, vmwar..."
4,Jonathan.pdf,18.666666666666668,"[3, 41, 12]",['national unive...,nus,jonathan@gmail.com,"ace pointer, ful...",Software Developer,"['ace pointer', ...",['ace pointer fu...,jonathanlim,+65-65824297,4.67,jonathan lim con...,3,python,nan,"master, bachelor",nan,aws,1,nan,1,nan,proposed,nan,1,nan,Nan,nan,Nan,1,nan,nan,nan,25.0,electrical engin...


In [59]:
newFinal = newFinal.replace("[({':]", "")

In [60]:
newFinal

,Filename,Average Change,Duration Per Job(m),Edu Desc,Education Level,Email Address,Experience,Job Applied,Job Desc,Job Role,Name,Phone Number,Total Experience in years,Word Cloud,dE_no,dataEn_skill,dataEngSkill,education,exp_no,good_to_have,gth_no,gth_skills,ld_no,leader,leadership,p_lvl,ra_no,rele_job,relevant_ability,relevant_skill,results_performance,rp_no,sg_sch,specialise,specialise_no,total_per,total_skills
0,JiaXin.pdf,5.0,"[9, 1]",['master of comp...,nus,jiaxinqiu@gmail.com,project 5c: post...,Data Engineer,['project 5c: po...,['responsible fo...,qiujiaxinobjectiv,+65-65829230,1.58,comprehensive ab...,9,"python, sql, java",28.929000000000002,"master, bachelor",5.0,image processing,1,0.41700000000000004,4,0.8,"responsible, tea...",0.235,2,0.0,"model, data anal...",1.294,"improve, drive, ...",3,5.0,master of comput...,20.0,61.675,"sales, video, po..."
1,Javier.pdf,24.5,"[18, 31]",['master of scie...,ntu,javier456@hotmai...,process integrat...,Data Engineer,['process integr...,['(sneaker hotsp...,jav,+65-65923456,4.08,javier tel 6592...,6,"python, sql",19.285999999999998,master,5.0,"numpy, machine l...",2,0.833,3,0.6,"cooperate, overs...",0.11800000000000001,1,5.0,Nan,0.647,"pinpoint, improv...",6,5.0,b.eng. in mechan...,20.0,56.483999999999995,"coding, root cau..."
2,LingPing.pdf,4.8,"[8, 8, 2, 2, 4]",['nanyang techno...,ntu,tanlingping@gmai...,data scientist i...,Data Engineer,['data scientist...,['data scientist...,tanlingpingping,+65-96240378,2.0,tan ling ping mo...,6,"python, sql",19.285999999999998,bachelor,5.0,Nan,1,0.41700000000000004,1,0.2,Nan,0.235,2,5.0,feature preparat...,1.294,predict,1,5.0,Not Relevant,0.0,36.431999999999995,"statistics, rese..."
3,Patricia.pdf,9.333333333333334,"[9, 7, 12]",['pamantasan lun...,Nan,patrciachen@gmai...,"airliquide.com, ...",Dev Ops,['airliquide.com...,['airliquide.com...,patriciachen,+65-89241058,2.33,patricia chen de...,4,"python, linux",nan,Nan,nan,"docker, shell, aws",3,nan,1,nan,spearheaded,nan,1,nan,Nan,nan,Nan,1,nan,nan,nan,25.0,"wordpress, vmwar..."
4,Jonathan.pdf,18.666666666666668,"[3, 41, 12]",['national unive...,nus,jonathan@gmail.com,"ace pointer, ful...",Software Developer,"['ace pointer', ...",['ace pointer fu...,jonathanlim,+65-65824297,4.67,jonathan lim con...,3,python,nan,"master, bachelor",nan,aws,1,nan,1,nan,proposed,nan,1,nan,Nan,nan,Nan,1,nan,nan,nan,25.0,electrical engin...


In [61]:
db1 = newFinal[['Filename','Name','Email Address','Phone Number','education','Job Applied','dataEn_skill','Total Experience in years','specialise','total_skills','Education Level','total_per']]

In [62]:
'''
set resume status to pending at start
'''
db1['res_status'] = 'Pending'
db1['total_skills'] = db1['total_skills'] + db1['dataEn_skill']
db1.drop(['dataEn_skill'],axis = 1,inplace  = True)

In [63]:
'''
set application_Date
'''
db1['application_date'] = '2020-01-29'
count = db1.shape[0]
countList = [x for x in range(1, count+1)]
db1['resume_id'] = countList

In [64]:
db1.columns = ['filename','name','email_add','phone_no','education_level','job_applied','total_experience','relevant_Job','total_skills','education_school','total_per','res_status','application_date','resume_id']

In [65]:
db1

,filename,name,email_add,phone_no,education_level,job_applied,total_experience,relevant_Job,total_skills,education_school,total_per,res_status,application_date,resume_id
0,JiaXin.pdf,qiujiaxinobjectiv,jiaxinqiu@gmail.com,+65-65829230,"master, bachelor",Data Engineer,1.58,master of comput...,"sales, video, po...",nus,61.675,Pending,2020-01-29,1
1,Javier.pdf,jav,javier456@hotmai...,+65-65923456,master,Data Engineer,4.08,b.eng. in mechan...,"coding, root cau...",ntu,56.483999999999995,Pending,2020-01-29,2
2,LingPing.pdf,tanlingpingping,tanlingping@gmai...,+65-96240378,bachelor,Data Engineer,2.0,Not Relevant,"statistics, rese...",ntu,36.431999999999995,Pending,2020-01-29,3
3,Patricia.pdf,patriciachen,patrciachen@gmai...,+65-89241058,Nan,Dev Ops,2.33,nan,"wordpress, vmwar...",Nan,25.0,Pending,2020-01-29,4
4,Jonathan.pdf,jonathanlim,jonathan@gmail.com,+65-65824297,"master, bachelor",Software Developer,4.67,nan,electrical engin...,nus,25.0,Pending,2020-01-29,5


In [66]:
db1 = db1[['resume_id','application_date','name','email_add','job_applied','res_status','phone_no','education_level','filename','total_skills','total_experience','relevant_Job','education_school','total_per']]
db1

,resume_id,application_date,name,email_add,job_applied,res_status,phone_no,education_level,filename,total_skills,total_experience,relevant_Job,education_school,total_per
0,1,2020-01-29,qiujiaxinobjectiv,jiaxinqiu@gmail.com,Data Engineer,Pending,+65-65829230,"master, bachelor",JiaXin.pdf,"sales, video, po...",1.58,master of comput...,nus,61.675
1,2,2020-01-29,jav,javier456@hotmai...,Data Engineer,Pending,+65-65923456,master,Javier.pdf,"coding, root cau...",4.08,b.eng. in mechan...,ntu,56.483999999999995
2,3,2020-01-29,tanlingpingping,tanlingping@gmai...,Data Engineer,Pending,+65-96240378,bachelor,LingPing.pdf,"statistics, rese...",2.0,Not Relevant,ntu,36.431999999999995
3,4,2020-01-29,patriciachen,patrciachen@gmai...,Dev Ops,Pending,+65-89241058,Nan,Patricia.pdf,"wordpress, vmwar...",2.33,nan,Nan,25.0
4,5,2020-01-29,jonathanlim,jonathan@gmail.com,Software Developer,Pending,+65-65824297,"master, bachelor",Jonathan.pdf,electrical engin...,4.67,nan,nus,25.0


In [67]:
db1

,resume_id,application_date,name,email_add,job_applied,res_status,phone_no,education_level,filename,total_skills,total_experience,relevant_Job,education_school,total_per
0,1,2020-01-29,qiujiaxinobjectiv,jiaxinqiu@gmail.com,Data Engineer,Pending,+65-65829230,"master, bachelor",JiaXin.pdf,"sales, video, po...",1.58,master of comput...,nus,61.675
1,2,2020-01-29,jav,javier456@hotmai...,Data Engineer,Pending,+65-65923456,master,Javier.pdf,"coding, root cau...",4.08,b.eng. in mechan...,ntu,56.483999999999995
2,3,2020-01-29,tanlingpingping,tanlingping@gmai...,Data Engineer,Pending,+65-96240378,bachelor,LingPing.pdf,"statistics, rese...",2.0,Not Relevant,ntu,36.431999999999995
3,4,2020-01-29,patriciachen,patrciachen@gmai...,Dev Ops,Pending,+65-89241058,Nan,Patricia.pdf,"wordpress, vmwar...",2.33,nan,Nan,25.0
4,5,2020-01-29,jonathanlim,jonathan@gmail.com,Software Developer,Pending,+65-65824297,"master, bachelor",Jonathan.pdf,electrical engin...,4.67,nan,nus,25.0


In [68]:
db1.to_csv('db.csv',index =False)

In [70]:
'''
Import csv file into mysql table
'''
import pymysql
import pandas as pd
import sys
def csv_to_mysql(load_sql, host, user, password):
    '''
    This function load a csv file to MySQL table according to
    the load_sql statement.
    '''
    try:
        con = pymysql.connect(host=host,
                                user=user,
                                password=password,
                                autocommit=True,
                                local_infile=1)
        print('Connected to DB: {}'.format(host))
        # Create cursor and execute Load SQL
        cursor = con.cursor()
        cursor.execute(load_sql)
        print('Succuessfully loaded the table from csv.')
        con.close()
       
    except Exception as e:
        print('Error: {}'.format(str(e)))
        sys.exit(1)

# Execution Example
load_sql = """LOAD DATA LOCAL INFILE 'db.csv' INTO TABLE resume_parser.candidate_info\
 FIELDS TERMINATED BY ',' ENCLOSED BY '"' IGNORE 1 LINES;"""
host = 'localhost'
user = 'root'
password = 'password'
csv_to_mysql(load_sql, host, user, password)

Error: (2003, "Can't connect to MySQL server on 'localhost' ([WinError 10061] No connection could be made because the target machine actively refused it)")


SystemExit: 1

In [ ]:
'''
Run this code in sql if mysql doesnt permit change
'''

# SHOW GLOBAL VARIABLES LIKE 'local_infile';
# SET GLOBAL local_infile = 'ON';
# SHOW GLOBAL VARIABLES LIKE 'local_infile';
